In [68]:
# load important packages
from __future__ import print_function, division
import numpy as np
import nibabel as nib
%matplotlib
import matplotlib.pyplot as plt
import re

Using matplotlib backend: MacOSX


In [69]:
from stimuli import events2neural

In [70]:
# %load subject_class.py
from __future__ import print_function, division
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

class subject(object):
    """try to organize data based on subject ex sub001"""
    def __init__(self,sub):
        # subject info ex: 'sub001'
        self.sub_id = sub

        # BOLD folder for this subject
        sub_BOLD_path = "../data/ds105/%s/BOLD/" %self.sub_id

        # anatomy folder for this subject
        sub_anatomy_file = "../data/ds105/%s/anatomy/highres001_brain.nii.gz" %self.sub_id
        
        # runfile_list: ['task001_run001', 'task001_run002'......]
        runfile_list = ['task001_run'+ i+ '.txt' for i in ['001','002','003','004','005','006','007','008','009','010','011','012']]
        runlist = ['run'+ i for i in ['001','002','003','004','005','006','007','008','009','010','011','012']]
        
        # TR info:
        self.TR = 2.5
        
        # load high resolution brain structure for this subject
        self.brain_img = nib.load(sub_anatomy_file)
        
        # load all bold image file for this subject
        self.run_img_result = {}
        for i in runlist:
            self.run_img_result[self.sub_id + '_' + i] = nib.load(sub_BOLD_path+'task001_'+i+'/bold.nii.gz')
        
        # all run keys:
        self.run_keys = self.run_img_result.keys()

        # shape of the BOLD data:
        self.BOLD_shape = self.run_img_result[self.run_keys[1]].shape
        
        # conditions setting: which condition is for which category
        self.condition_key_file = open("../data/ds105/models/model001/condition_key.txt")
        condition_list = self.condition_key_file.readlines()
        condition = re.compile(r'(cond\d+) (\w+)')
        result = {}
        for item in condition_list:
            for match in condition.finditer(item):
                result[match.group(1)] = match.group(2)
        self.condition_key = result
        
        # condition files for each objects for each run
        sub_condition_path = "../data/ds105/%s/model/model001/onsets/" %self.sub_id
        self.conditions = {}
        for i in runfile_list:
            for j in self.condition_key.keys():
                self.conditions[i[8:14]+'-'+self.condition_key[j]] = sub_condition_path + i[:-4]+'/'+j+'.txt'
                   
                



In [71]:
sub1 = subject("sub001")

In [72]:
sub1.BOLD_shape

(40, 64, 64, 121)

In [73]:
sub1.TR

2.5

In [74]:
sub1.brain_img

In [75]:
sub1.condition_key

{'cond001': 'house',
 'cond002': 'scrambledpix',
 'cond003': 'cat',
 'cond004': 'shoe',
 'cond005': 'bottle',
 'cond006': 'scissors',
 'cond007': 'chair',
 'cond008': 'face'}

In [76]:
sub1_condition_files = sub1.conditions

In [77]:
sub1_condition_files

{'run001-bottle': '../data/ds105/sub001/model/model001/onsets/task001_run001/cond005.txt',
 'run001-cat': '../data/ds105/sub001/model/model001/onsets/task001_run001/cond003.txt',
 'run001-chair': '../data/ds105/sub001/model/model001/onsets/task001_run001/cond007.txt',
 'run001-face': '../data/ds105/sub001/model/model001/onsets/task001_run001/cond008.txt',
 'run001-house': '../data/ds105/sub001/model/model001/onsets/task001_run001/cond001.txt',
 'run001-scissors': '../data/ds105/sub001/model/model001/onsets/task001_run001/cond006.txt',
 'run001-scrambledpix': '../data/ds105/sub001/model/model001/onsets/task001_run001/cond002.txt',
 'run001-shoe': '../data/ds105/sub001/model/model001/onsets/task001_run001/cond004.txt',
 'run002-bottle': '../data/ds105/sub001/model/model001/onsets/task001_run002/cond005.txt',
 'run002-cat': '../data/ds105/sub001/model/model001/onsets/task001_run002/cond003.txt',
 'run002-chair': '../data/ds105/sub001/model/model001/onsets/task001_run002/cond007.txt',
 'ru

In [78]:
sub1.sub_id

'sub001'

In [84]:
# %load get_object_neural.py

def get_object_neural(sub_ID ,condition_dict, object_name, TR, n_vox, check = 0, ):
    """
    get object neural array for specific object from all runs, odd runs or even runs
    
    parameters
    ----------
    condition_dict: dictionary of all condition files
    object_name: desired object name: 'house','scrambledpix','cat','shoe','bottle','scissors','chair','face'
    TR: TR for fMRI
    n_vox: time course of fMRI
    check: 1= odd runs, 2=even runs, others = all runs
    
    output
    -------
    a dictionary of run name : (neural array, path of the bold image) for that run
    ex:
    
    {
    'run001-bottle-neural': (array([ 0.,  0.,  0.,  0.,  0., ...]), bold path)
    'run002-bottle-neural': (array([ 0.,  0.,  0.,  0.,  0., ...]), bold path)
    }
    
    
    """
    
    condition_dict_key = condition_dict.keys()
    result = {}
    for i in condition_dict_key:
        if check == 1:
            if object_name in i:
                if int(i[3:6])%2 ==1:
                    result[i+'-neural'] = (events2neural(condition_dict[i], TR, n_vox), "../data/ds105/%s/BOLD/task001_"% sub_ID + i[:6]+"/bold.nii.gz")
        elif check == 2:
            if object_name in i: 
                if int(i[3:6])%2 ==0:
                    result[i+'-neural'] = (events2neural(condition_dict[i], TR, n_vox), "../data/ds105/%s/BOLD/task001_"% sub_ID + i[:6]+"/bold.nii.gz")
        else:
            if object_name in i:
                result[i+'-neural'] = (events2neural(condition_dict[i], TR, n_vox), "../data/ds105/%s/BOLD/task001_"%sub_ID + i[:6]+"/bold.nii.gz")
    
    return result




In [85]:
cat_odd_neural = get_object_neural(sub1.sub_id, sub1.conditions,"cat" , sub1.TR, sub1.BOLD_shape[-1], check=1)

In [86]:
cat_odd_neural

{'run001-cat-neural': (array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
          0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.]),
  '../data/ds105/sub001/BOLD/task001_run001/bold.nii.gz'),
 'run003-cat-neural': (array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 

In [87]:
cat_odd_neural['run001-cat-neural']

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.]),
 '../data/ds105/sub001/BOLD/task001_run001/bold.nii.gz')

In [90]:
# %load ObtainData.py
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

def pull3D(DictionaryOfVectors):
	Three_D_Data = []
	for key, value in DictionaryOfVectors.iteritems():
		vector = value[0]
		fileLocation = value[1]
		img = nib.load(fileLocation)
		data = img.get_data()
		for i in range(0, len(vector)):
			if vector[i] == 1:
				Three_D_Data.append(data[:, :, :, i])
	return Three_D_Data

def ShowCortex(ListOf3dImages):
	for image in ListOf3dImages:
		zDimension = np.shape(image)[2]
		for i in range(0, zDimension):
			plt.imshow(image[:, :, i], cmap = "gray")
			plt.show()

# You'll have to guess the Z value from the images show in the previous function
def generateEvenlySpacedZpoints(zValue):
	return np.linspace(start = 0, stop = zValue, num = zValue)

In [91]:
aa =pull3D(cat_odd_neural)

In [92]:
len(aa)

14